# Configurando o Ambiente e baixando os dados

In [2]:
pip install -u basedosdados


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [3]:
import basedosdados as bd

# Respondendo as perguntas

## 1. Quantos chamados foram abertos no dia 01/04/2023?

In [5]:
query ='''
SELECT COUNT(*) AS total_chamados
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) = '2023-04-01';
'''

In [6]:
df = bd.read_sql(query, billing_project_id="projeto-estudando-gcp")
df.head()

Downloading: 100%|██████████|


,total_chamados
0,1903


## 2. Qual o tipo de chamado que teve mais chamados abertos no dia 01/04/2023?

In [7]:
query ='''
SELECT tipo, COUNT(*) AS total
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) = '2023-04-01'
GROUP BY tipo
ORDER BY total DESC
LIMIT 1;
'''

In [8]:
df = bd.read_sql(query, billing_project_id="projeto-estudando-gcp")
df.head()

Downloading: 100%|██████████|


,tipo,total
0,Estacionamento irregular,373


## 3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [11]:
query ='''
SELECT b.nome AS bairro, COUNT(*) AS total
FROM `datario.adm_central_atendimento_1746.chamado` c
JOIN `datario.dados_mestres.bairro` b
  ON c.id_bairro = b.id_bairro
WHERE DATE(c.data_inicio) = '2023-04-01'
GROUP BY bairro
ORDER BY total DESC
LIMIT 3;
'''

In [12]:
df = bd.read_sql(query, billing_project_id="projeto-estudando-gcp")
df.head()

Downloading: 100%|██████████|


,bairro,total
0,Campo Grande,124
1,Tijuca,96
2,Barra da Tijuca,60


## 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [13]:
query ='''
SELECT b.subprefeitura, COUNT(*) AS total
FROM `datario.adm_central_atendimento_1746.chamado` c
JOIN `datario.dados_mestres.bairro` b
  ON c.id_bairro = b.id_bairro
WHERE DATE(c.data_inicio) = '2023-04-01'
GROUP BY b.subprefeitura
ORDER BY total DESC
LIMIT 1;
'''

In [14]:
df = bd.read_sql(query, billing_project_id="projeto-estudando-gcp")
df.head()

Downloading: 100%|██████████|


,subprefeitura,total
0,Zona Norte,534


## 5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura? Se sim, por que?

In [15]:
query ='''
SELECT COUNT(*) AS chamados_sem_bairro
FROM `datario.adm_central_atendimento_1746.chamado` c
LEFT JOIN `datario.dados_mestres.bairro` b
  ON c.id_bairro = b.id_bairro
WHERE DATE(c.data_inicio) = '2023-04-01'
  AND b.id_bairro IS NULL;
'''

In [16]:
df = bd.read_sql(query, billing_project_id="projeto-estudando-gcp")
df.head()

Downloading: 100%|██████████|


,chamados_sem_bairro
0,131


Explicação possível: Pode acontecer por inconsistência no preenchimento do campo id_bairro, ou porque o local informado pelo cidadão não pôde ser georreferenciado corretamente.
Na base temos 131 chamadas abertas sem associação a bairro e subprefeitura. Se elas não possuem vínculo com um id_bairro ou subprefeitura, então elas podem corresponder a tipos independentes do espaço geográfico. Um exemplo seriam as demandas do atendimento, como  notificações sobre erros e problemas em serviços digitais da prefeitura que não estão vinculadas a um local específico.

## 6. Quantos chamados de "Perturbação do sossego" foram abertos entre 01/01/2022 e 31/12/2024?

In [17]:
query ='''
SELECT COUNT(*) AS chamados_sem_bairro
FROM `datario.adm_central_atendimento_1746.chamado` c
LEFT JOIN `datario.dados_mestres.bairro` b
  ON c.id_bairro = b.id_bairro
WHERE DATE(c.data_inicio) = '2023-04-01'
  AND b.id_bairro IS NULL;
'''

In [18]:
df = bd.read_sql(query, billing_project_id="projeto-estudando-gcp")
df.head()

Downloading: 100%|██████████|


,chamados_sem_bairro
0,131


## 7. Chamados de perturbação durante eventos específicos (Reveillon, Carnaval, Rock in Rio):

In [19]:
query ='''
SELECT c.*
FROM `datario.adm_central_atendimento_1746.chamado` c
JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
  ON DATE(c.data_inicio) BETWEEN DATE(e.data_inicial) AND DATE(e.data_final)
WHERE c.id_subtipo = '5071'
  AND DATE(c.data_inicio) BETWEEN '2022-01-01' AND '2024-12-31';
'''

In [20]:
df = bd.read_sql(query, billing_project_id="projeto-estudando-gcp")
df.head()

Downloading: 100%|██████████|


,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,data_real_diagnostico,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao
0,17684293,1,2022-09-10 22:20:02,2022-09-10 22:35:59,93,3,49627,303,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,NaT,<NA>,D,F,No prazo,Encerrado,Não atendido,None,0,2022-09-01
1,17664461,1,2022-09-04 19:26:30,2022-09-13 10:42:12,14,1,70235,518,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,NaT,<NA>,D,F,Fora do prazo,Encerrado,Não atendido,None,0,2022-09-01
2,17661848,1,2022-09-03 05:37:04,2022-09-11 08:25:43,14,1,70235,518,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,NaT,<NA>,D,F,Fora do prazo,Encerrado,Não atendido,None,0,2022-09-01
3,17685630,1,2022-09-11 23:11:52,2022-09-19 10:41:49,24,2,66076,28,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,NaT,<NA>,D,F,Fora do prazo,Encerrado,Não atendido,None,0,2022-09-01
4,17685603,1,2022-09-11 22:27:49,2022-09-19 10:39:13,64,3,81034,232,70,GM-RIO - Guarda Municipal do Rio de Janeiro,...,NaT,<NA>,D,F,Fora do prazo,Encerrado,Não atendido,None,0,2022-09-01


## 8. Quantos chamados desse subtipo foram abertos em cada evento?

In [36]:
query ='''
SELECT e.evento, COUNT(*) AS total_chamados
FROM `datario.adm_central_atendimento_1746.chamado` c
JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
  ON DATE(c.data_inicio) BETWEEN DATE(e.data_inicial) AND DATE(e.data_final)
WHERE c.id_subtipo = '5071'
  AND DATE(c.data_inicio) BETWEEN '2022-01-01' AND '2024-12-31'
GROUP BY e.evento
order by total_chamados desc
'''

In [35]:
df = bd.read_sql(query, billing_project_id="projeto-estudando-gcp")
df.head()

Downloading: 100%|██████████|


,evento,total_chamados
0,Réveillon,147
1,Rock in Rio,946
2,Carnaval,252


## 9. Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [38]:
query ='''
SELECT
  e.evento,
  COUNT(*) AS total_chamados,
  DATE_DIFF(DATE(e.data_final), DATE(e.data_inicial), DAY) + 1 AS duracao_dias,
  COUNT(*) / (DATE_DIFF(DATE(e.data_final), DATE(e.data_inicial), DAY) + 1) AS media_diaria
FROM `datario.adm_central_atendimento_1746.chamado` c
JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
  ON DATE(c.data_inicio) BETWEEN DATE(e.data_inicial) AND DATE(e.data_final)
WHERE c.id_subtipo = '5071'
  AND DATE(c.data_inicio) BETWEEN '2022-01-01' AND '2024-12-31'
GROUP BY e.evento, e.data_inicial, e.data_final
ORDER BY media_diaria DESC
LIMIT 1;
'''

In [39]:
df = bd.read_sql(query, billing_project_id="projeto-estudando-gcp")
df.head()

Downloading: 100%|██████████|


,evento,total_chamados,duracao_dias,media_diaria
0,Rock in Rio,550,4,137.5


## 10. Comparação entre médias diárias durante eventos vs. média geral

In [40]:
query ='''
-- Média geral
WITH media_geral AS (
  SELECT
    COUNT(*) / DATE_DIFF('2024-12-31', '2022-01-01', DAY) AS media_diaria_geral
  FROM `datario.adm_central_atendimento_1746.chamado`
  WHERE id_subtipo = '5071'
    AND DATE(data_inicio) BETWEEN '2022-01-01' AND '2024-12-31'
),

-- Média por evento
media_evento AS (
  SELECT
    e.evento,
    COUNT(*) / (DATE_DIFF(DATE(e.data_final), DATE(e.data_inicial), DAY) + 1) AS media_diaria_evento
  FROM `datario.adm_central_atendimento_1746.chamado` c
  JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
    ON DATE(c.data_inicio) BETWEEN DATE(e.data_inicial) AND DATE(e.data_final)
  WHERE c.id_subtipo = '5071'
    AND DATE(c.data_inicio) BETWEEN '2022-01-01' AND '2024-12-31'
  GROUP BY e.evento, e.data_inicial, e.data_final
)

SELECT *
FROM media_evento, media_geral;

'''

In [41]:
df = bd.read_sql(query, billing_project_id="projeto-estudando-gcp")
df.head()

Downloading: 100%|██████████|


,evento,media_diaria_evento,media_diaria_geral
0,Carnaval,63.0,51.858447
1,Réveillon,49.0,51.858447
2,Rock in Rio,137.5,51.858447
3,Rock in Rio,132.0,51.858447
